# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import warnings
import pickle
import gzip
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
#, FeatureUnion, make_pipeline, make_union
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
#, make_scorer, accuracy_score, jaccard_score, hamming_loss
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/kbosko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/kbosko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
X = df['message']
Y = df.iloc[:, 4:]
category_names = df.columns[4:]

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = pipeline.predict(X_test)

In [9]:
for i, col in enumerate(Y.columns):
    print(i, col)
    print(classification_report(Y_test.to_numpy()[:, i], Y_pred[:, i]))

0 related
              precision    recall  f1-score   support

           0       0.65      0.36      0.46      1266
           1       0.81      0.94      0.87      3938
           2       0.62      0.20      0.30        40

    accuracy                           0.79      5244
   macro avg       0.69      0.50      0.54      5244
weighted avg       0.77      0.79      0.77      5244

1 request
              precision    recall  f1-score   support

           0       0.89      0.99      0.93      4349
           1       0.86      0.39      0.54       895

    accuracy                           0.89      5244
   macro avg       0.87      0.69      0.74      5244
weighted avg       0.88      0.89      0.87      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50   

/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
classification_labels = pipeline.predict(['We are more than 50 people sleeping on the street. Please, help us find tent, food.'])[0]
classification_results = dict(zip(df.columns[4:], classification_labels))
classification_results

{'related': 1,
 'request': 1,
 'offer': 0,
 'aid_related': 1,
 'medical_help': 0,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'child_alone': 0,
 'water': 0,
 'food': 1,
 'shelter': 1,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 0,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 0,
 'floods': 0,
 'storm': 0,
 'fire': 0,
 'earthquake': 0,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 1}

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__

In [12]:
parameters = {
    'vect__stop_words': ('english', None),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0)
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1), vect__stop_words=english 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  vect__max_df=0.5, vect__ngram_range=(1, 1), vect__stop_words=english, score=0.234, total=  25.6s
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1), vect__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.6s remaining:    0.0s
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  vect__max_df=0.5, vect__ngram_range=(1, 1), vect__stop_words=english, score=0.230, total=  24.9s
[CV] vect__max_df=0.5, vect__ngram_range=(1, 1), vect__stop_words=english 


...

/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=english, score=0.225, total=  41.3s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=english 


/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=english, score=0.237, total=  41.6s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None 
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None, score=0.217, total=  38.3s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None 
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None, score=0.220, total=  37.8s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None 
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 2), vect__stop_words=None, score=0.229, total=  38.2s


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 20.1min finished
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [18]:
print(cv.best_params_, cv.best_score_)

{'vect__max_df': 0.5, 'vect__ngram_range': (1, 2), 'vect__stop_words': 'english'} 0.23998664886515353


In [19]:
for i, col in enumerate(Y.columns):
    print(i, col)
    print(classification_report(Y_test.to_numpy()[:, i], Y_pred[:, i]))

0 related
              precision    recall  f1-score   support

           0       0.58      0.49      0.53      1266
           1       0.84      0.88      0.86      3938
           2       0.22      0.23      0.22        40

    accuracy                           0.78      5244
   macro avg       0.55      0.53      0.54      5244
weighted avg       0.77      0.78      0.78      5244

1 request
              precision    recall  f1-score   support

           0       0.90      0.97      0.93      4349
           1       0.77      0.45      0.57       895

    accuracy                           0.88      5244
   macro avg       0.83      0.71      0.75      5244
weighted avg       0.87      0.88      0.87      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5021
           1       0.50      0.00      0.01       223

    accuracy                           0.96      5244
   macro avg       0.73      0.50      0.49      5244
weighted avg       0.94      0.96      0.94      5244

28 weather_related
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      3806
           1       0.83      0.54      0.65      1438

    accuracy                           0.84      5244
   macro avg       0.84      0.75      0.78      5244
weighted avg       0.84      0.84      0.83      5244

29 floods
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4833
           1       0.89      0.35      0.50       411

    accuracy                           0.95      5244
   macro avg       0.92      0.67      0.74      5244
weighted avg       0.94      0.95      0.93  

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
#specifying the best parameters that were determined in the previous gridsearch step
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.5, ngram_range=(1, 2), stop_words='english')),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
    ])

In [31]:
#trying out different ML algrorithms 
parameters = {
    'clf': (MultiOutputClassifier(SGDClassifier(random_state = 42)),
            MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
}

In [32]:
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

It turns out that we can't run SGDClassifier on the original dataset. The reason is that there's a column "child_alone" that has no positive instances! That's why I'm dropping here this category and do the train-test split again.

In [9]:
df_nochildalone = df.drop(['child_alone'], axis=1)

X_nochildalone = df_nochildalone['message']
Y_nochildalone = df_nochildalone.iloc[:, 4:]
X_train_nca, X_test_nca, Y_train_nca, Y_test_nca = train_test_split(X_nochildalone, Y_nochildalone, test_size=0.2, random_state=42, shuffle=True)

In [34]:
cv.fit(X_train_nca, Y_train_nca)
Y_pred = cv.predict(X_test_nca)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf=MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight=None,
                                              early_stopping=False, epsilon=0.1,
                                              eta0=0.0, fit_intercept=True,
                                              l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=1000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=42, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=Fal

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf=MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight=None,
                                              early_stopping=False, epsilon=0.1,
                                              eta0=0.0, fit_intercept=True,
                                              l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=1000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=42, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                      n_jobs=None), score=0.268, tota

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.5s remaining:    0.0s
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf=MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight=None,
                                              early_stopping=False, epsilon=0.1,
                                              eta0=0.0, fit_intercept=True,
                                              l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=1000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=42, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                      n_jobs=None), score=0.271, tota

...

/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf=MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators='warn',
                                                       n_jobs=None,
       

/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf=MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators='warn',
                                                       n_jobs=None,
       

[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  2.5min finished
/Users/kbosko/anaconda3/envs/pysyft/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [35]:
print(cv.best_params_, cv.best_score_)

{'clf': MultiOutputClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                              class_weight=None,
                                              early_stopping=False, epsilon=0.1,
                                              eta0=0.0, fit_intercept=True,
                                              l1_ratio=0.15,
                                              learning_rate='optimal',
                                              loss='hinge', max_iter=1000,
                                              n_iter_no_change=5, n_jobs=None,
                                              penalty='l2', power_t=0.5,
                                              random_state=42, shuffle=True,
                                              tol=0.001,
                                              validation_fraction=0.1,
                                              verbose=0, warm_start=False),
                      n_jobs=None)} 0.27207705512111385

In [37]:
for i, col in enumerate(Y_nochildalone.columns):
    print(i, col)
    print(classification_report(Y_test_nca.to_numpy()[:, i], Y_pred[:, i]))

0 related
              precision    recall  f1-score   support

           0       0.72      0.33      0.45      1266
           1       0.81      0.96      0.88      3938
           2       0.00      0.00      0.00        40

    accuracy                           0.80      5244
   macro avg       0.51      0.43      0.44      5244
weighted avg       0.78      0.80      0.77      5244

1 request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4349
           1       0.82      0.56      0.66       895

    accuracy                           0.90      5244
   macro avg       0.87      0.77      0.80      5244
weighted avg       0.90      0.90      0.90      5244

2 offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50   

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5127
           1       0.67      0.07      0.12       117

    accuracy                           0.98      5244
   macro avg       0.82      0.53      0.56      5244
weighted avg       0.97      0.98      0.97      5244

33 other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4968
           1       0.67      0.01      0.01       276

    accuracy                           0.95      5244
   macro avg       0.81      0.50      0.49      5244
weighted avg       0.93      0.95      0.92      5244

34 direct_report
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      4223
           1       0.75      0.41      0.53      1021

    accuracy                           0.86      5244
   macro avg       0.81      0.69      0.72      5244
weighted avg       0.85      0.86      0

In [5]:
#specifying the best parameters that were determined in the previous gridsearch step
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, max_df=0.5, ngram_range=(1, 2), stop_words='english')),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SGDClassifier(random_state = 42)))
    ])

### 9. Export your model as a pickle file

In [38]:
with gzip.open('model.p.gz', 'wb') as gzipped_f:
    # Pickle the trained pipeline using the highest protocol available.
    pickled = pickle.dumps(pipeline)
    gzipped_f.write(pickled)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
#Below is the code I tried with FeatureUnion to add information from 'genre' column. The code works. 
#However, I realized that I won't be able to automate message categorization as it is currently implemented in the app
#because I will need to specify genre next to the message as input (--> redesign the app)
#Something to try in the future!

In [ ]:
# # load data from database
# engine = create_engine('sqlite:///data/DisasterResponse.db')
# df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
# X = df[['message', 'genre']]
# Y = df.iloc[:, 4:]
# category_names = df.columns[4:]

In [ ]:
#from sklearn.base import TransformerMixin

# class DataFrameColumnExtracter(TransformerMixin):

#     def __init__(self, column):
#         self.column = column

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return X[self.column]

In [ ]:
# class FeatureReshaper(TransformerMixin):

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return X.to_numpy().reshape(-1, 1)

In [ ]:
#from sklearn.preprocessing import OneHotEncoder

# nlp_pipe = make_pipeline(
#     DataFrameColumnExtracter('message'), 
#     CountVectorizer(tokenizer=tokenize),
#     TfidfTransformer(),
# )

# dummy_pipe = make_pipeline(
#     DataFrameColumnExtracter('genre'),
#     FeatureReshaper(),
#     OneHotEncoder()
# )

# feature_union = make_union(nlp_pipe, dummy_pipe)

In [ ]:
# pipeline = Pipeline([
#     ('features', feature_union),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])